In [2]:
import scipy.io as sio
import scipy
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import time
from tensorflow.contrib.slim import fully_connected as fc
from tensorflow.python.ops.parallel_for.gradients import jacobian
import pandas as pd
import seaborn as sn
import cv2
import random
import h5py
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


In [3]:
filepath = 'train.mat'
arrays = {}
f = h5py.File(filepath)
for k, v in f.items():
    arrays[k] = np.array(v)

In [4]:
train = arrays["train"]
np.save('train.npy',train)

In [3]:
train = np.load('train.npy')

In [4]:

test = sio.loadmat('test.mat')
PHI_ALL = sio.loadmat('PHI_ALL.mat')
LOGPERM_MEAN = sio.loadmat('logmean.mat')

test = np.array(test["test"]).transpose()
PHI_ALL = np.array(PHI_ALL["PHI_ALL"]).transpose()
LOGPERM_MEAN = np.array(LOGPERM_MEAN["logmean"]).transpose()
input_dim = 192000
num_sample = 4000
totals = 8
maxiter = 4
kpers = 25
totalk = totals*kpers
PHIMIX = sio.loadmat('PHIMIX.mat')
PHIMIX = np.array(PHIMIX["PHIMIX"]).transpose()
k=[25,25,25,25,25,25,25,25];
n1=0
n2=0
PHI = np.zeros((np.sum(k),192000))
print(train.shape)
print(test.shape)
print(PHI_ALL.shape)
print(LOGPERM_MEAN.shape)
print(PHI.shape)
for nf in range(totals):
    n2 =k[nf]+n2
    PHI[n1: n2] = PHI_ALL[nf*100:nf*100+k[nf]]
    n1 = k[nf]+n1
    
k=[25,25,25,25,25,25,25,25];
n1=0
n2=0
PHIn = np.zeros((np.sum(k),192000))
for nf in range(totals):
    n2 =k[nf]+n2
    PHIn[n1: n2] = PHI_ALL[nf*100:nf*100+k[nf]]
    n1 = k[nf]+n1

(3200, 192000)
(800, 192000)
(800, 192000)
(1, 192000)
(200, 192000)


In [12]:
class CNN(object):

    def __init__(self, conv1n):

        tf.reset_default_graph()
        self.conv1n = conv1n
        self.build()
        self.sess = tf.InteractiveSession(config=config)
        self.sess.run(tf.global_variables_initializer())

    # Build the netowrk and the loss functions
    def build(self):
        self.x = tf.placeholder(
            name='x', dtype=tf.float32, shape=[None, input_dim])
        self.labels = tf.placeholder(
            name='labels', dtype=tf.int64, shape=[None, 8])
        self.learning_rate = tf.placeholder(name = 'learning_rate',dtype = tf.float32,shape = None)
        self.drate = tf.placeholder(name = 'drate',dtype = tf.float32,shape = None)
        self.onehotgrad = tf.placeholder(name = 'onehotgrad',dtype = tf.float32,shape = [None,8])
        self.l1c = tf.placeholder(name = 'l1c',dtype = tf.float32,shape = None)
        self.l2c = tf.placeholder(name = 'l2c',dtype = tf.float32,shape = None)
        inp0 = tf.nn.dropout(self.x,rate = self.drate)
        inp = tf.reshape(inp0,[-1,40,120,40,1])
        self.conv1 = tf.contrib.layers.conv3d(inp,self.conv1n,3,stride=1,
                                 padding='Same',activation_fn=tf.nn.relu,scope = 'Conv1')
        mp1 = tf.contrib.layers.avg_pool3d(self.conv1,[2,2,2])
        d1 = tf.nn.dropout(mp1,rate = self.drate)
        self.conv2 = tf.contrib.layers.conv3d(d1,self.conv1n*2,3,stride=1,
                                 padding='Same',activation_fn=tf.nn.relu,scope = 'Conv2')
        self.mp2 = tf.contrib.layers.avg_pool3d(self.conv2,[2,2,2])
        d2 = tf.nn.dropout(self.mp2,rate = self.drate)
        
        self.conv3 = tf.contrib.layers.conv3d(d2,self.conv1n*4,2,stride=1,
                                 padding='Same',activation_fn=tf.nn.relu,scope = 'Conv3')

        self.mp3 = tf.contrib.layers.avg_pool3d(self.conv3,[2,2,2])
        d3 = tf.nn.dropout(self.mp3,rate = self.drate)
        self.conv4 = tf.contrib.layers.conv3d(d3,self.conv1n*8,2,stride=1,
                                 padding='Same',activation_fn=tf.nn.relu,scope = 'Conv4')
        
        self.mp4 = tf.contrib.layers.max_pool3d(self.conv4,[2,2,2])
        
        print(self.mp4.shape)
        flate1 = tf.reshape(self.mp4,[-1,2*7*2*self.conv1n*8])
        self.logits =fc(flate1, 8, scope='FC1',activation_fn=None)
        print(self.logits.shape)
        print(self.labels.shape)
        self.l2w = sum(tf.nn.l2_loss(var) for var in tf.trainable_variables() if not 'biases' in var.name)
        self.l2b = sum(tf.nn.l2_loss(var) for var in tf.trainable_variables() if not 'weights' in var.name)
        self.l1w = sum(tf.reduce_sum(tf.abs(var)) for var in tf.trainable_variables() if not 'biases' in var.name)
        self.l1b = sum(tf.reduce_sum(tf.abs(var)) for var in tf.trainable_variables() if not 'weights' in var.name)
        self.loss = tf.losses.softmax_cross_entropy(self.labels, self.logits)
        #self.total_loss = self.loss
        self.total_loss = self.loss  + self.l2c*self.l2w + self.l2c*self.l2b +self.l1c*self.l1w +self.l1c*self.l1b
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss)
        
        self.losses = {
            'loss': self.loss,
            'total_loss': self.total_loss,
            'L1':self.l1w,
            'L2':self.l2w
        }     
        self.prob = tf.nn.softmax(self.logits)
        self.correct = tf.equal(tf.argmax(self.prob,1), tf.argmax(self.labels, 1))
        self.accuracy=tf.reduce_mean(tf.cast(self.correct, tf.float32))
        self.y_c = tf.reduce_sum(tf.multiply(self.logits,self.onehotgrad)+tf.constant(1e-5),axis = 1)
        self.grad = tf.gradients(self.y_c,self.mp2)[0]
        
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x,labels,learning_rate,drate,l1c,l2c):
        _, losses = self.sess.run(
            [self.train_op, self.losses],
            feed_dict={self.x: x,self.labels: labels, self.learning_rate:learning_rate,self.drate: drate
                      ,self.l1c:l1c, self.l2c:l2c}
        )
        return losses

    # x -> x_hat
    def predictprob(self, x):
        drate = 0
        prob = self.sess.run(self.prob, feed_dict={self.x: x,self.drate: drate})
        
        return prob
    def predictclass(self, x):
        drate = 0
        logits = self.sess.run(self.logits, feed_dict={self.x: x,self.drate: drate})
        return pclass
    def calaccuracy(self,x,label):
        drate = 0
        accuracy = self.sess.run(self.accuracy, feed_dict={self.x: x,self.labels:label,self.drate: drate})
        
        return accuracy
    def caljacob(self,x,label,l1c,l2c):
        drate = 0
        jacob = self.sess.run(jacobian(self.total_loss,self.x), feed_dict={self.x:x,self.labels:label,self.drate:drate
                                                                          ,self.l1c:l1c, self.l2c:l2c})
        return jacob
    def gradcam(self,x,nlabel):
        drate = 0
        onehotgrad = np.zeros((1,5))
        onehotgrad[0,nlabel] = 1
        gradcam = self.sess.run(self.grad,feed_dict={self.x:x,self.drate:drate,self.onehotgrad:onehotgrad})
        return gradcam
    def actmp2(self,x):
        drate = 0
        actmp2 = self.sess.run(self.mp4,feed_dict = {self.x:x,self.drate:drate})
        return actmp2

In [13]:
def trainer(
    model_object,convl1n,
    learning_rate=1e-4, batch_size=64, train = train,test = test,
    num_epochs=100, log_step=10, rnv = 1, rbv = 1, rnc = 0.5, rbc = 0.5,drate = 0,l1c = 0,l2c = 0,nscenario = 8,savename = "tf_models/SPCNN.ckpt"):
    model = model_object(
      convl1n
    )
    trainlabels = np.zeros((400*nscenario,nscenario))
    testlabels = np.zeros((100*nscenario,nscenario))
    for iscenario in range(nscenario):
        trainlabels[iscenario*400:(iscenario+1)*400,iscenario] = 1
    for iscenario in range(nscenario):
        testlabels[iscenario*100:(iscenario+1)*100,iscenario] = 1
    for epoch in range(num_epochs):
        start_time = time.time()
        seq = np.random.permutation(400*nscenario)
        traini = train[seq]
        labelsi = trainlabels[seq]
        for iter in range((400*nscenario) // batch_size):
            batch = traini[iter*batch_size:(iter+1)*batch_size]
            batchl = labelsi[iter*batch_size:(iter+1)*batch_size]
            losses = model.run_single_step(batch,batchl,learning_rate,drate =drate,l1c = l1c, l2c = l2c)
        end_time = time.time()
        if epoch % log_step == 0:
            acc = 0
            for nset in range(16):
                acc = acc + model.calaccuracy(train[nset*200:(nset+1)*200],trainlabels[nset*200:(nset+1)*200])
            acc = acc/16
            log_str = '[Epoch: {}] '.format(epoch)
            log_str += 'Accuracy: {:.3f}, '.format(acc)
            for k, v in losses.items():
                log_str += '{}: {:.3f}, '.format(k, v)
            log_str += '({:.3f}sec/epoch)'.format(end_time - start_time)
            print(log_str)
            acc = 0
            for nset in range(16):
                acc = acc + model.calaccuracy(test[nset*50:(nset+1)*50],testlabels[nset*50:(nset+1)*50])
            acc = acc/16
            log_str = 'Accuracy: {:.3f}, '.format(acc)
            print(log_str)
        if epoch % 50== 0:
            learning_rate = learning_rate*0.95
    saver = tf.train.Saver()
    model_path = saver.save(model.sess, savename)
    acc = 0
    for nset in range(16):
        acc = acc + model.calaccuracy(train[nset*200:(nset+1)*200],trainlabels[nset*200:(nset+1)*200])
    acc = acc/16
    log_str = '[Epoch: {}] '.format(epoch)
    log_str += 'Accuracy: {:.3f}, '.format(acc)
    for k, v in losses.items():
        log_str += '{}: {:.3f}, '.format(k, v)
    log_str += '({:.3f}sec/epoch)'.format(end_time - start_time)
    print(log_str)
    acc = 0
    for nset in range(16):
        acc = acc + model.calaccuracy(test[nset*50:(nset+1)*50],testlabels[nset*50:(nset+1)*50])
    acc = acc/16
    log_str = 'Accuracy: {:.3f}, '.format(acc)
    print(log_str)
    
    print("Model saved in %s" % model_path)
    print('Done')
    return model

In [34]:
model.sess.close()

In [35]:
model = trainer(CNN, 2,num_epochs=20, learning_rate=5e-4, batch_size=50,
                train = train,rnc = 2, rbc = 3,drate = 0, l1c = 0.000001, l2c = 0.0001,savename = "tf_models/CNN3D.ckpt")

(?, 2, 7, 2, 16)
(?, 8)
(?, 8)
[Epoch: 0] Accuracy: 0.865, loss: 0.461, total_loss: 0.464, L1: 367.747, L2: 19.482, (8.288sec/epoch)
Accuracy: 0.862, 
[Epoch: 10] Accuracy: 1.000, loss: 0.000, total_loss: 0.003, L1: 377.417, L2: 21.066, (8.616sec/epoch)
Accuracy: 1.000, 
[Epoch: 19] Accuracy: 1.000, loss: 0.000, total_loss: 0.002, L1: 370.303, L2: 20.474, (8.564sec/epoch)
Accuracy: 1.000, 
Model saved in tf_models/CNN3D.ckpt
Done


In [14]:
model = CNN(2)
saver = tf.train.Saver()
saver.restore(model.sess, "tf_models/CNN3D.ckpt")

Instructions for updating:
Colocations handled automatically by placer.
(?, 2, 7, 2, 16)
(?, 8)
(?, 8)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from tf_models/CNN3D.ckpt
